Step 1: Create conda environment

In [ ]:
%%bash
# Some users experienced issues on Ubuntu with an AMD CPU
# Install libopenblas-dev (issue #115, thanks WindWing)
# sudo apt-get install libopenblas-dev
export TORCH_CUDA_ARCH_LIST="6.0 6.1 6.2 7.0 7.2 7.5 8.0 8.6"
conda env create -f environment_small.yml
conda activate mask_3d

Step 2: Install cuda-toolkit and correct gxx version

In [ ]:
%%bash
#conda install nvidia/label/cuda-11.3.1::cuda-toolkit
conda install conda-forge::cudatoolkit=11.3.1
conda install conda-forge::'gxx[version=">=5,<10.2.1"]'

Step 3: Install pip requirements

In [ ]:
%%bash
pip install pip==23.3
PIP_CONSTRAINT=constraints.txt pip install -r requirements.txt --no-deps --no-cache

Step 4: Install pyviz3d

In [ ]:
%pip install pyviz3d==0.2.28

Step 5: Install torch, torch scatter and detectron2

In [ ]:
pip3 install torch==1.12.1+cu113 torchvision==0.13.1+cu113 --extra-index-url https://download.pytorch.org/whl/cu113
pip3 install torch-scatter -f https://data.pyg.org/whl/torch-1.12.1+cu113.html
#pip3 install 'git+https://github.com/facebookresearch/detectron2.git@710e7795d0eeadf9def0e7ef957eea13532e34cf' --no-deps
cd third_party
git clone https://github.com/facebookresearch/detectron2.git
python -m pip install -e detectron2

Step 6: Install the MinkowskiEngine

In [ ]:
%%bash
git clone --recursive "https://github.com/NVIDIA/MinkowskiEngine"
cd MinkowskiEngine
conda install -c conda-forge blas=*=openblas
git checkout 02fc608bea4c0549b0a7b00ca1bf15dee4a0b228
python setup.py install --force_cuda --blas=openblas

Step 7: Setup ScanNet

In [ ]:
%%bash
cd ..
git clone https://github.com/ScanNet/ScanNet.git
cd ScanNet/Segmentator
git checkout 3e5726500896748521a6ceb81271b0f5b2c0e7d2
make

Step 8: Setup PointNet2

In [ ]:
%%bash
cd ../../pointnet2
python setup.py install

Step 9: Install pytorch-lightning

In [ ]:
%%bash
cd ../../
pip3 install pytorch-lightning==1.7.2 --no-deps

If your unable to run the model after step 9, check the version of hydra-core and omegaconf and update them

In [ ]:
%%bash
pip install hydra-core==1.0.5
pip install omegaconf==2.2

# 2. Datasets

## Semantic Terrain Points Labeling - Synthetic 3D (STPLS3D)

Download from: https://drive.google.com/file/d/1ZGS5WDL3th7_OsC7E4kAfvOXjCY9ugyR/view

Extract and put content into path: data/raw/stpls3d

Split into train and validation dataset: 
- trainSplit = [1, 2, 3, 4, 6, 7, 8, 9, 11, 12, 13, 14, 16, 17, 18, 19, 21, 22, 23, 24]    
- valSplit = [5, 10, 15, 20, 25] 

In [7]:
!python -m datasets.preprocessing.stpls3d_preprocessing preprocess --data_dir="data/raw/stpls3d" --save_dir="data/processed/stpls3d"

2025-01-09 09:29:44.788 | INFO     | datasets.preprocessing.base_preprocessing:preprocess:47 - Tasks for train: 20
[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   4 out of  20 | elapsed:    8.7s remaining:   34.8s
[Parallel(n_jobs=16)]: Done   7 out of  20 | elapsed:    9.1s remaining:   16.9s
[Parallel(n_jobs=16)]: Done  10 out of  20 | elapsed:    9.5s remaining:    9.5s
[Parallel(n_jobs=16)]: Done  13 out of  20 | elapsed:    9.7s remaining:    5.2s
[Parallel(n_jobs=16)]: Done  16 out of  20 | elapsed:   10.2s remaining:    2.6s
[Parallel(n_jobs=16)]: Done  20 out of  20 | elapsed:   12.2s finished
2025-01-09 09:29:57.002 | INFO     | datasets.preprocessing.base_preprocessing:preprocess:47 - Tasks for validation: 5
[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   2 out of   5 | elapsed:    9.4s remaining:   14.1s
[Parallel(n_jobs=16)]: Done   3 out of   5 | elapsed:  

# 3. Training and testing

## Semantic Terrain Points Labeling - Synthetic 3D (STPLS3D)

Download checkpoint from Github Repo and extract into path "checkpoints/stpls3d/stpls3d_val.ckpt"

In [ ]:
%%bash
./scripts/stpls3d/stpls3d_val.sh